In [76]:
import pandas as pd
import numpy as np

In [77]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import  SimpleImputer
from sklearn.tree import DecisionTreeClassifier

In [78]:
df = pd.read_csv('train.csv')
df.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
526,527,1,2,"Ridsdale, Miss. Lucy",female,50.0,0,0,W./C. 14258,10.5000,NaN,S
752,753,0,3,"Vande Velde, Mr. Johannes Joseph",male,33.0,0,0,345780,9.5000,NaN,S
874,875,1,2,"Abelson, Mrs. Samuel (Hannah Wizosky)",female,28.0,1,0,P/PP 3381,24.0000,NaN,C
604,605,1,1,"Homer, Mr. Harry (""Mr E Haven"")",male,35.0,0,0,111426,26.5500,NaN,C
137,138,0,1,"Futrelle, Mr. Jacques Heath",male,37.0,1,0,113803,53.1000,C123,S
191,192,0,2,"Carbines, Mr. William",male,19.0,0,0,28424,13.0000,NaN,S
443,444,1,2,"Reynaldo, Ms. Encarnacion",female,28.0,0,0,230434,13.0000,NaN,S
89,90,0,3,"Celotti, Mr. Francesco",male,24.0,0,0,343275,8.0500,NaN,S
263,264,0,1,"Harrison, Mr. William",male,40.0,0,0,112059,0.0000,B94,S
628,629,0,3,"Bostandyeff, Mr. Guentcho",male,26.0,0,0,349224,7.8958,NaN,S


In [79]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Dropping the columns less required. inplace = True, used to drop orignally from orignal dataframe.

In [80]:
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'],inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [81]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=['Survived']), df['Survived'], test_size=0.2, random_state=4)

In [82]:
si_age = SimpleImputer() #will replace the missing value with mean
si_embarked = SimpleImputer(strategy='most_frequent') #will replace with the most frequently repeated value

x_train_age = si_age.fit_transform(x_train[['Age']])
x_train_embarked = si_embarked.fit_transform(x_train[['Embarked']])

x_test_age = si_age.fit_transform(x_test[['Age']])
x_test_embarked = si_embarked.fit_transform(x_test[['Embarked']])

In [83]:
ohe_sex = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

x_train_sex = ohe_sex.fit_transform(x_train[['Sex']])
x_train_embarked = ohe_embarked.fit_transform(x_train_embarked)

x_test_sex = ohe_sex.fit_transform(x_test[['Sex']])
x_test_embarked = ohe_embarked.fit_transform(x_test_embarked)

In [84]:
x_train_rem = x_train.drop(columns=['Sex', 'Embarked', 'Age'])
x_test_rem = x_test.drop(columns=['Sex', 'Embarked', 'Age'])

In [85]:
x_train_trans = np.concatenate((x_train_rem, x_train_sex, x_train_age, x_train_embarked),axis=1)
x_test_trans = np.concatenate((x_test_rem, x_test_sex, x_test_age, x_test_embarked),axis=1)

In [86]:
clf = DecisionTreeClassifier()
clf.fit(x_train_trans, y_train)

DecisionTreeClassifier()

In [87]:
y_pred = clf.predict(x_test_trans)
y_pred.shape

(179,)

In [88]:
y_test.shape

(179,)

In [89]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8324022346368715

In [90]:
import pickle
pickle.dump(ohe_sex,open('models/ohe_sex.pkl','wb'))
pickle.dump(ohe_embarked,open('models/ohe_embarked.pkl','wb'))
pickle.dump(clf,open('models/clf.pkl','wb'))